# Read this first 

* each comment and text line added here is important and relevant , dont skim through just the code
* if the comment asks you to revisit some concepts/modules , do so
* this is not complete code on its own , you need to fill in your own code to eventually start building a model

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import numpy as np

In [2]:
datafile_train="E:/data/Hazard_train.csv"
datafile_test="E:/data/Hazard_test_share.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [3]:
bd_train.head()

,Id,Hazard,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
0,83642,2,10,23,4,N,K,Y,B,D,...,2,37,1,11,2,Y,N,E,2,1
1,92530,6,3,18,2,B,H,N,B,B,...,2,34,1,12,7,Y,N,C,3,3
2,86131,4,5,20,4,B,K,N,B,B,...,1,40,1,17,4,Y,N,D,1,1
3,88666,1,4,7,5,N,H,N,B,B,...,2,37,1,11,6,Y,N,D,2,1
4,11122,21,10,21,6,N,C,Y,B,C,...,2,40,1,6,6,N,N,D,5,5


In [4]:
bd_test.head()

,Id,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,T1_V9,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
0,67943,9,18,2,N,B,Y,B,C,D,...,2,34,1,14,4,Y,N,E,3,3
1,38287,12,20,5,N,C,Y,D,B,D,...,1,40,1,9,3,Y,N,E,1,1
2,51951,14,18,1,N,K,Y,B,D,E,...,2,25,1,1,4,Y,N,E,2,1
3,40476,19,16,1,W,H,Y,B,B,D,...,2,25,1,1,7,Y,N,D,2,11
4,101190,12,21,2,B,K,Y,B,B,E,...,2,22,1,20,6,Y,N,C,2,1


In [6]:
cat_cols=bd_train.select_dtypes(['object']).columns

In [7]:
cat_cols

Index(['T1_V4', 'T1_V5', 'T1_V6', 'T1_V7', 'T1_V8', 'T1_V9', 'T1_V11',
       'T1_V12', 'T1_V15', 'T1_V16', 'T1_V17', 'T2_V3', 'T2_V5', 'T2_V11',
       'T2_V12', 'T2_V13'],
      dtype='object')

In [8]:
bd_train['data']='train'
bd_test['data']='test'


In [9]:
all_data=pd.concat([bd_train,bd_test],0,sort=False)

In [10]:
all_data['data'].unique()

array(['train', 'test'], dtype=object)

In [11]:
for col in cat_cols:
    k=all_data[col].value_counts()
    cats=k[k>=100].index[:-1]
    
    for cat in cats:
        name=col+'_'+cat
        all_data[name]=(all_data[col]==cat).astype(int)
        
    del all_data[col]

In [12]:
all_data.shape

(50999, 94)

In [13]:
all_data.head()

,Id,Hazard,T1_V1,T1_V2,T1_V3,T1_V10,T1_V13,T1_V14,T2_V1,T2_V2,...,T2_V5_A,T2_V5_B,T2_V5_C,T2_V5_D,T2_V11_Y,T2_V12_N,T2_V13_E,T2_V13_A,T2_V13_C,T2_V13_D
0,83642,2.0,10,23,4,2,10,0,52,14,...,0,0,1,0,1,1,1,0,0,0
1,92530,6.0,3,18,2,8,10,3,39,14,...,1,0,0,0,1,1,0,0,1,0
2,86131,4.0,5,20,4,7,15,1,66,10,...,0,0,1,0,1,1,0,0,0,1
3,88666,1.0,4,7,5,8,15,3,48,11,...,0,1,0,0,1,1,0,0,0,1
4,11122,21.0,10,21,6,12,10,1,42,11,...,1,0,0,0,0,1,0,0,0,1


In [14]:
x_train=all_data.drop(['Id','Hazard','data'],1)[all_data['data']=='train']
y_train=all_data['Hazard'][all_data['data']=='train']

In [15]:
x_train.shape, y_train.shape

((40799, 91), (40799,))

In [16]:
x_test=all_data.drop(['Id','Hazard','data'],1)[all_data['data']=='test']

In [20]:
pip install xgboost

  Using cached xgboost-1.6.2-py3-none-win_amd64.whl (125.4 MB)
Note: you may need to restart the kernel to use updated packages.


Keyring is skipped due to an exception: 'keyring.backends'


In [21]:
#import xgboost
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

In [22]:
parameters = {'objective':['count:poisson'],
              "learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4]}

In [23]:
model= XGBRegressor()

In [24]:
grid_xgb = RandomizedSearchCV(model,param_distributions=parameters, cv=10,n_iter=10,scoring='neg_mean_absolute_error',verbose=20,n_jobs=-1)
grid_xgb.fit(x_train,y_train,verbose = 1)


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   55.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   55.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  1

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, gamma=None,
                                          gpu_id=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=No...
                                          reg_alpha=None, ...),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_d

In [25]:
grid_xgb.best_params_

{'objective': 'count:poisson',
 'min_child_weight': 5,
 'max_depth': 6,
 'learning_rate': 0.05,
 'gamma': 0.1,
 'colsample_bytree': 0.4}

In [27]:
RG=XGBRegressor(**{'objective': 'count:poisson',
 'min_child_weight': 5,
 'max_depth': 6,
 'learning_rate': 0.05,
 'gamma': 0.1,
 'colsample_bytree': 0.4})

In [28]:
XGReg=RG.fit(x_train,y_train)

In [29]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [30]:
err_xgb=mean_absolute_error(y_train, y_pred_train, multioutput='raw_values')

NameError: name 'y_pred_train' is not defined

In [31]:
from sklearn.linear_model import Lasso

# you should try other algos for improvement in performance 
# the target here is not really suitable for linear regression or regression cost in general
# as it takes always positive integer values 
# you should try PoissonRegressor available with sklearn .23 and onwards 
# if that is not available , you can also make use of objective :'count:poisson' with xgboost
# you can also try HistGradientBoostingRegressor with loss='poisson'


In [32]:
params={'alpha':np.linspace(0.01,10,100)}

In [33]:
from sklearn.model_selection import GridSearchCV

In [34]:
model=Lasso(fit_intercept=True)

In [35]:
grid_search=GridSearchCV(model,cv=10,scoring='neg_mean_absolute_error',
                         param_grid=params,n_jobs=-1,verbose=10)

# you can also make use of mean_poisson_deviance as scoring , this ignores non-positive outcomes
# it might not be avalaible with your version

In [36]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:   

GridSearchCV(cv=10, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': array([ 0.01      ,  0.11090909,  0.21181818,  0.31272727,  0.41363636,
        0.51454545,  0.61545455,  0.716...
        7.57818182,  7.67909091,  7.78      ,  7.88090909,  7.98181818,
        8.08272727,  8.18363636,  8.28454545,  8.38545455,  8.48636364,
        8.58727273,  8.68818182,  8.78909091,  8.89      ,  8.99090909,
        9.09181818,  9.19272727,  9.29363636,  9.39454545,  9.49545455,
        9.59636364,  9.69727273,  9.79818182,  9.89909091, 10.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolu

In [37]:
grid_search.best_estimator_

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
# you can see that best estimator is at the edge of the alpha range that we tried . You
# can expand on that, if you are using Lasso

In [38]:
abs(grid_search.best_score_)

2.748947530559763

In [39]:
submissions=pd.DataFrame({'Hazard':grid_search.predict(x_test)})

In [40]:
submissions.to_csv('sample_submission.csv',index=False)